# Introduction to tf-shell

To get started, `pip install tf-shell`. tf-shell has a few modules, the one used
in this notebook is `tf_shell`.

In [1]:
import tf_shell
import tensorflow as tf

2024-10-29 18:41:45.349763: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 18:41:45.375597: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


First, set up some low level parameters for the BGV encryption scheme and
generate a secret key. If you don't want to set these parameters yourself, see
the `intro_with_auto_params.ipynb` notebook.

- `log_n` coresponds to the ring degree.
- `main_moduli` are a product of primes, less than 60 bits, that define the main
modulus. tf-shell uses RNS representation over each of these moduli.
- `scaling_factor` is used to encode fractional numbers, e.g. the number 1.5
with a scaling factor of 2 will be represented as 3.

In [2]:
context = tf_shell.create_context64(
    log_n=10,
    main_moduli=[8556589057, 8388812801],
    plaintext_modulus=40961,
    scaling_factor=3,
    seed="test_seed",
)

secret_key = tf_shell.create_key64(context)

INFO: Generating key


Next, encrypt something. tf-shell treats the outer dimension of all tensors as
the packing dimension of the BGV scheme. This is for efficiency purposes and, in
short, means the first dimension of all tensors is 2^log_n as defined above.

The cost of operating on a ciphertext (e.g. addition), regardless of how "full"
the first dimension is, is the same.

In [3]:
tf_data = tf.random.uniform([context.num_slots, 2], dtype=tf.float32, maxval=10)
print(f"The first 3 elements of the data are {tf_data[:3, 0]}")

enc = tf_shell.to_encrypted(tf_data, secret_key, context)

The first 3 elements of the data are [4.8077097 6.9659996 0.500679 ]


Next, perform some homomorphic operations on the ciphertexts. tf-shell can
perform addition, subtraction, and multiplication on ciphertexts, shell
plaintexts, and tensorflow tensors.

In [4]:
# Ciphertext . scalar
ct_scalar_add = enc + 3
ct_scalar_mul = enc * 3

# Ciphertext . tensorflow tensor
three = tf.constant(3, dtype=tf.float32)
ct_tf_add = enc + three
ct_tf_mul = enc * three

# Ciphertext . plaintext
three_repeated = tf.ones([context.num_slots, 1], dtype=tf.float32) * 3
three_repeated = tf_shell.to_shell_plaintext(three_repeated, context)
ct_pt_add = enc + three_repeated
ct_pt_mul = enc * three_repeated

# ciphertext . ciphertext
ct_ct_add = enc + enc
ct_ct_mul = enc * enc

Finally, let's decrypt the results using the secret key.

In [5]:
dec = tf_shell.to_tensorflow(enc, secret_key)
add = tf_shell.to_tensorflow(ct_ct_add, secret_key)
mul = tf_shell.to_tensorflow(ct_ct_mul, secret_key)

print(f"enc:       {dec[:3, 0]}")
print(f"enc + enc: {add[:3, 0]}")
print(f"enc * enc: {mul[:3, 0]}")

enc:       [4.6666665 7.        0.6666667]
enc + enc: [ 9.333333  14.         1.3333334]
enc * enc: [21.777779   49.          0.44444445]


## Scaling Factors
tf-shell keeps track of scaling factors, so you don't have to.

In this example, we'll encrypt a ciphertext with scaling factor 3 (defined in
the context above) and multiply it by a plaintext. The resulting ciphertext will
have a scaling factor of 9.

In [6]:
a = tf.constant([.33], dtype=tf.float32)
enc_a = tf_shell.to_encrypted(a, secret_key, context)

enc_mul = enc_a * .33
mul = tf_shell.to_tensorflow(enc_mul, secret_key)
print(f".33 * .33 = {mul[0]}")
print(f"Scaling factor: {enc_mul.scaling_factor}")

.33 * .33 = 0.1111111119389534
Scaling factor: 9


Say we perform another multiplication, the scaling factor will be 27.

In [7]:
enc_mul_mul = enc_mul * .33
mul_mul = tf_shell.to_tensorflow(enc_mul_mul, secret_key)
print(f".33 * .33 * .33 = {mul_mul[0]}")
print(f"Scaling factor: {enc_mul_mul.scaling_factor}")

.33 * .33 * .33 = 0.03703703731298447
Scaling factor: 27


The output scaling factor of multiplication is the product of the scaling
factors of the operands e.g. sf=3 \* sf=9 -> sf=27.

Additive operations, on the other hand, have their scaling factors matched to
the LCM (least common multiple) of each. Here we'll add a ciphertext with
scaling factor 9 to a ciphertext with scaling factor 27. The result will have a
scaling factor of 27.

In [8]:
enc_mul_mul = enc_mul + enc_mul_mul
mul_mul = tf_shell.to_tensorflow(enc_mul_mul, secret_key)
print(f"(.33 * .33) + (.33 * .33 * .33) = {mul_mul[0]}")
print(f"Scaling factor: {enc_mul_mul.scaling_factor}")

(.33 * .33) + (.33 * .33 * .33) = 0.14814814925193787
Scaling factor: 27


## Modulus Switching
`tf-shell` supports modulus switching and will keep track of the moduli just
like it does with scaling factors.

In [ ]:
a = tf.constant([.33], dtype=tf.float32)
enc_a = tf_shell.to_encrypted(a, secret_key, context)

mod_reduced_a = tf_shell.mod_reduce_tensor64(enc_a)
reduced_sum = enc_a + mod_reduced_a  # enc_a is mod_reduced before the addition.

print(f"Sum: {tf_shell.to_tensorflow(reduced_sum, secret_key)[0]}")
print(f"Level of arg1: {enc_a.level} arg2: {mod_reduced_a.level} sum: {reduced_sum.level}")


Sum: 0.6666666865348816
Level of arg1: 2 arg2: 1 sum: 1
